In [9]:
import numpy as np
import scipy
import torch.optim as optim
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from tqdm import notebook

import dwave
import dimod
from dwave.samplers import SimulatedAnnealingSampler
from dwave.system.composites import FixedEmbeddingComposite
from dwave.system.samplers import DWaveSampler

In [2]:
class ExactAnnealing_QE:
    def __init__(self, H, num):
        self.H         = H
        self.num       = num
        self.sampleset = []

    def Eigenvector(self):
        alpha      = 0.8
        H          = self.H
        N          = H.shape[0]
        I          = np.identity(N)
        Binary_H   = Binary_Hamiltonian(H, N)
        Confi      = dimod.ExactSolver().sample(Binary_H).first
        X          = np.array([Confi[0][n] + 0. for n in range(N)])
        Lowest_E   = Energy(self.H, X)
        Energy_set = np.zeros(self.num)

        for i in range(self.num):
            Energy_set[i] = Lowest_E
            Diagonal      = Lowest_E * I
            H_prime       = self.H - Diagonal
            onsite        = 2.0 * onsite_E(H_prime, X)
            onsite        = onsite * I
            H_prime       = H_prime + onsite

            Binary_H      = Binary_Hamiltonian(H_prime, N)
            Confi         = dimod.ExactSolver().sample(Binary_H).first
            Y             = np.array([Confi[0][n] + 0. for n in range(N)])
            aubxiliary    = np.random.uniform(alpha, 2.0 - alpha , N)
            aubxiliary    = 1.0
            X             = X + alpha * Y * aubxiliary
            Lowest_E      = Energy(self.H, X)
            if Energy_set[i] < Lowest_E:
                Lowest_E = Energy_set[i]
                alpha    = 0.2 * alpha
            else:
                self.sampleset.append(Energy_set[i])
                alpha    = 0.5
        X = X / np.sqrt(X @ X)
        return X

    def Eigenvalue(self):
        return min(self.sampleset)

    def Convergence(self):
        return self.sampleset

In [3]:
def Binary_Hamiltonian(H, N):
    linear    = {}
    quadratic = {}
    for n in range(N):
        linear[n] = H[n][n]
    for n in range(N):
        for m in range(N):
            quadratic[(n,m)] = H[n,m]
    H = dimod.BinaryQuadraticModel(linear, quadratic, 0, vartype='SPIN')
    return H

def Energy(H, X):
    return np.dot(X, H@X)/ np.dot(X, X)

def onsite_E(H, X):
    return H @ X

In [44]:
iteration   = 100
Mat     = np.array([[1,2,3,4,-5,3,-2,4],[2,5,6,7,2,-6,1,-7],[3,6,10,8,3,-5,-4,2],
              [4,7,8,15,1,6,8,9],[-5,2,3,1,20,3,-5,6],[3,-6,-5,6,3,-14,2,-5],
              [-2,1,-4,8,-5,2,-7,4],[4,-7,2,9,6,-5,4,2]])
N = 10
np.random.seed(1)
Mat          = np.random.random((N,N))
Mat          = Mat + Mat.T
eival, eivec = scipy.linalg.eigh(Mat)
eivec        = eivec.T

In [45]:
iteration = 200
QE = ExactAnnealing_QE(Mat, iteration)
Eigenvector = QE.Eigenvector()
Eigenvalue  = QE.Eigenvalue()

In [29]:
Eigenvector

array([-0.26758049,  0.26758049,  0.08704426, -0.29337138, -0.26758049,
        0.68023474, -0.26758049,  0.39653494])

In [46]:
np.abs(Eigenvalue-eival[0]) / np.abs(eival[0])

0.019025173880577097

In [12]:
Eigenvalue

-21.806907378335946

In [26]:
QE.Convergence()

[-13.25,
 -15.299180327868852,
 -16.062500000000004,
 -19.09761163032191,
 -21.16979949874687,
 -21.75077881619938,
 -21.806907378335946]